In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from sqlalchemy import create_engine
import sqlite3

In [73]:
path = '/Applications/Driver/chromedriver-mac-arm64/chromedriver'
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(service=Service(executable_path=path), options=chrome_options)

In [74]:
url='https://www.trivago.com/en-INR/lm/hotels-new-york-new-york-state?search=200-14734;dr-20241007-20241010-s;rc-1-1'

In [70]:
def is_element_present(htl, by, value):
    try:
        htl.find_element(by, value)
        return True
    except NoSuchElementException:
        return False

In [ ]:
hotel_names = []
hotel_prices = []
hotel_stars = []
hotel_rating = []

wait = WebDriverWait(driver, 15)
driver.get(url)

for i in range(2,12):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(15)
    all_hotels = driver.find_elements(By.CSS_SELECTOR, "div[class='slideout-container_wrapper__cVquW slideout-container_withShadow__FqUic']")

    print(len(set(all_hotels)))

    for hotel in all_hotels:
        rating = "0"
        name = hotel.find_element(By.CSS_SELECTOR,"button[class='item-name_button__1e5cV truncate_truncate__vCzPM']").find_element(By.CSS_SELECTOR,"span[itemprop='name']")
        price = hotel.find_element(By.CSS_SELECTOR,"p[data-testid='price-per-stay']").find_element(By.CSS_SELECTOR,"strong")
        stars = len(hotel.find_elements(By.CSS_SELECTOR,"span[data-testid='star']"))
        if is_element_present(hotel, By.CSS_SELECTOR,"span[itemprop='ratingValue']"):
            rating = hotel.find_element(By.CSS_SELECTOR,"span[itemprop='ratingValue']").text
    
        if name and price and stars and rating != "0":
            hotel_names.append(name.text)
            hotel_prices.append(price.text)
            hotel_stars.append(stars)
            hotel_rating.append(rating)

    print(len(hotel_names))
    time.sleep(5)
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,"span[class='Icon_wrapper__B6IoS Icon_m__qOPUh Pagination_icon__2IRJL']"))).click()
    
driver.quit()
print(hotel_names)

# Create a DataFrame and save to CSV
trivago = pd.DataFrame({
    'Hotel Name': hotel_names,
    'Price': hotel_prices,
    'Star': hotel_stars,
    'Rating': hotel_rating
})
print(trivago)

## PUSHING DATA TO CSV

In [79]:
trivago.to_csv('trivago_nyc.csv', index=False)

## PUSHING DATA TO DATABASE

In [ ]:
# Establish a connection to SQLite database
engine = create_engine('sqlite:///hotel_booking_nyc.db', echo=True)

# Insert DataFrame into the database table
trivago.to_sql('trivago_hotel_table', con=engine, if_exists='replace', index=False)

# Verify the data insertion
conn = sqlite3.connect('hotel_booking_nyc.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM trivago_hotel_table")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()